Übung: 3 VMs zu einem Kubernetes Cluster verbinden (k3s in kubevirt - Variante)
--------------------------------------------------

K3s bietet eine einfache und effiziente Möglichkeit, Kubernetes-Cluster lokal oder in Produktionsumgebungen zu betreiben. 

Das Hinzufügen von Nodes zu einem bestehenden K3s-Cluster erfolgt über den Befehl `curl -sfL https://get.k3s.io`. 

- - -

Um die Vorgänge zu visualisieren starten wird Kubernetes Dashboard und scrollen rechts herunter zu Cluster -> Nodes

Wählt dazu nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.


In [ ]:
! echo "https://"$(cat ~/work/server-ip)":30443"

---

###  Worker Nodes zur Control Plane hinzufügen (k3s + kubevirt Variante)

* Dazu holen wir uns zuerst den Token von der Control Plane Node
* Installieren k3s auf der Worker Node
* Joinen die Worker Node mit der Control Plane Node

In [ ]:
%%bash
SERVER_WG_IP="$(ip -4 addr show wg0 | awk '/inet /{print $2}' | cut -d/ -f1)"
SERVER_NAT_IP="$(hostname -I | awk '{print $1}')"

K3S_URL="https://${SERVER_WG_IP:-$SERVER_NAT_IP}:6443"
TOKEN="$(sudo cat /var/lib/rancher/k3s/server/node-token)"

join_node() {
  NODE="$1"

  echo "🔗 [INFO] Verbinde $NODE"

  ssh "$NODE" bash -s <<EOF
set -e

echo "🧠 [INFO] Prüfe Netzwerk auf \$(hostname)"

WG_IP=\$(ip -4 addr show wg0 2>/dev/null | awk '/inet /{print \$2}' | cut -d/ -f1)

BASE_ARGS="agent --server $K3S_URL --token $TOKEN"

if [ -n "\$WG_IP" ]; then
  echo "🔐 [INFO] WireGuard erkannt (\$WG_IP)"
  K3S_ARGS="\$BASE_ARGS --node-ip=\$WG_IP --flannel-iface=wg0"
else
  echo "🌍 [INFO] Kein WireGuard – NAT Join"
  K3S_ARGS="\$BASE_ARGS"
fi

if systemctl is-active --quiet k3s-agent; then
  echo "⚠️ [WARN] k3s-agent läuft bereits – überspringe"
  exit 0
fi

echo "▶️ [INFO] Starte Join: \$K3S_ARGS"

curl -sfL https://get.k3s.io | INSTALL_K3S_EXEC="\$K3S_ARGS" sh -
EOF
}

# === Nodes hier auflisten ===
join_node duk-lernvirt-vm-1
join_node duk-lernvirt-vm-2

In [ ]:
! kubectl get nodes 

- - -

### Zentraler Persistenter Speicher

Damit alle Kubernetes Nodes (Master + Worker) den gleichen zentralen Persistenten Speicher verwenden, wurder vorher auf dem Master ein NFS Share /data eingerichtet.

In [ ]:
! cat /etc/exports

Damit müssen sich jetzt noch die Worker Nodes verbinden

In [ ]:
%%bash
SERVER_WG_IP="$(ip -4 addr show wg0 | awk '/inet /{print $2}' | cut -d/ -f1)"
SERVER_NAT_IP="$(hostname -I | awk '{print $1}')"

SERVER_IP="${SERVER_WG_IP:-$SERVER_NAT_IP}"
ssh duk-lernvirt-vm-1 -- "sudo umount /data ; sudo mount -t nfs $SERVER_IP:/data /data; df -h | grep /data"

In [ ]:
%%bash
SERVER_WG_IP="$(ip -4 addr show wg0 | awk '/inet /{print $2}' | cut -d/ -f1)"
SERVER_NAT_IP="$(hostname -I | awk '{print $1}')"

SERVER_IP="${SERVER_WG_IP:-$SERVER_NAT_IP}"
ssh duk-lernvirt-vm-2 -- "sudo umount /data ; sudo mount -t nfs $SERVER_IP:/data /data; df -h | grep /data"

Die entsprechende Dateiablage, sogenannte PersistentVolumes (PV) in Kubernetes, wurde beim Installieren von Kubernetes eingerichtet. Das PersistentVolumes zeigt auf das Verzeichnis `/data`, siehe Eintrag `hostPath`.

Diese Einrichtung ermöglicht die Trennung von Speicher und Container-Lebenszyklus, sodass Daten über Container-(Pod)-Neustarts und -Löschungen hinaus bestehen bleiben.

In [ ]:
%%bash
kubectl get pv -o go-template='{{ range .items }}
apiVersion: v1
kind: PersistentVolume
metadata:
  name: {{ .metadata.name }}
spec:
  capacity:
    storage: {{ .spec.capacity.storage }}
  accessModes:
  {{- range .spec.accessModes }}
    - {{ . }}
  {{- end }}
  storageClassName: {{ .spec.storageClassName }}
  persistentVolumeReclaimPolicy: {{ .spec.persistentVolumeReclaimPolicy }}
  volumeMode: {{ .spec.volumeMode }}

  {{- if .spec.hostPath }}
  hostPath:
    path: {{ .spec.hostPath.path }}
    type: {{ .spec.hostPath.type }}
  {{- end }}

  {{- if .spec.claimRef }}
  claimRef:
    namespace: {{ .spec.claimRef.namespace }}
    name: {{ .spec.claimRef.name }}
  {{- end }}
status:
  phase: {{ .status.phase }}
---
{{ end }}'


Im Kubernetes Dashboard finden wir den Eintrag in dem wir rechts herunterscrollen auf Cluster -> PersistentVolumes

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":30443"